In [3]:
# townweather.py
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# 檢查是否有鄉鎮市區代碼檔
if not os.path.isfile('district.csv'):
    df = pd.read_excel('712693030RPKUP4RX.xlsx', header=3)
    df.drop(columns=['縣市代碼', '村里代碼', '村里名稱', '村里代碼', '村里代碼.1'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df.to_csv('district.csv', encoding='big5', index=False)


In [4]:

dftown = pd.read_csv('district.csv', encoding='big5')  #區鄉鎮名稱代碼資料
town = "桃園市桃園區"
dfs = dftown[(dftown['縣市名稱']==town[:3]) & (dftown['區鄉鎮名稱']==town[3:])]


In [5]:
if len(dfs) > 0:  #區鄉鎮名稱存在
    town_no = str(dfs.iloc[0,1])
    url = 'https://www.cwb.gov.tw/V8/C/W/Town/MOD/3hr/' + town_no + '_3hr_PC.html'  #三日預報網頁
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    print(soup.find_all("tr")[4])


<tr>
<th id="PC3_AT">體感溫度</th>
<td headers="PC3_AT PC3_D1 PC3_D1H00"><span class="tem-C is-active">31</span><span class="tem-F is-hidden">88</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H03"><span class="tem-C is-active">30</span><span class="tem-F is-hidden">86</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H06"><span class="tem-C is-active">29</span><span class="tem-F is-hidden">84</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H09"><span class="tem-C is-active">34</span><span class="tem-F is-hidden">93</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H12"><span class="tem-C is-active">38</span><span class="tem-F is-hidden">100</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H15"><span class="tem-C is-active">38</span><span class="tem-F is-hidden">100</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H18"><span class="tem-C is-active">33</span><span class="tem-F is-hidden">91</span></td>
<td headers="PC3_AT PC3_D1 PC3_D1H21"><span class="tem-C is-active">31</span><span class="tem-F is-hidden">88</sp

In [6]:

    soup_data = {"時間":[],"日期":[],"天氣狀況":[],'溫度(℃)':[],'溫度(℉)':[],"體感溫度(℃)":[],"體感溫度(℉)":[]\
                 ,"降雨機率":[],"相對溼度":[],"蒲福風級":[],"風速(m/s)":[],"風向":[],"舒適度":[]}
    # 整理日期時間欄
    for t in soup.find('tr',class_="time").find_all("span"):
        soup_data["時間"].append(t.text)
    for d in soup.find('tr').find_all("th", headers="PC3_D"):
        if d.get("id") == "PC3_D4" : soup_data["日期"] += [d.text]
        else : soup_data["日期"] += [d.text]*8

In [7]:
    # 整理天氣示意圖欄
    for img in soup.find_all('tr')[2].find_all("img"):
        soup_data["天氣狀況"].append(img.get("title"))

In [8]:
    # 整理溫度欄
    for c in soup.find_all("tr")[3].find_all("span"):
        if c.get("class")[0] == "tem-C" : soup_data['溫度(℃)'].append(c.text)
        else : soup_data['溫度(℉)'].append(c.text)
    # 整理體感溫度欄
    for f in soup.find_all("tr")[4].find_all("span"):
        if f.get("class")[0] == "tem-C" : soup_data['體感溫度(℃)'].append(f.text)
        else : soup_data['體感溫度(℉)'].append(f.text)

In [9]:
    # 整理降雨機率欄
    for t in soup.find_all("tr")[5].find_all("td"):
        if t.get("headers")[2] == "PC3_D4H00" : soup_data['降雨機率'] += [t.text]
        else : soup_data['降雨機率'] += [t.text]*2

In [10]:
    # 整理相對溼度欄
    for t in soup.find_all("tr")[6].find_all("td"):
        soup_data['相對溼度'].append(t.text)   
    # 整理蒲福風級 
    for w1 in soup.find_all("tr")[7].find_all("td"):
        soup_data['蒲福風級'].append(t.text)
    # 整理風速
    for s in soup.find_all("tr")[8].find_all("td"):
        soup_data['風速(m/s)'].append(s.text)
    # 整理風向
    for s in soup.find_all("tr")[9].find_all("td"):
        soup_data['風向'].append(s.text)
     # 整理舒適度
    for s in soup.find_all("tr")[10].find_all("td"):
        soup_data['舒適度'].append(s.text)


In [11]:
   # pandas讀取表格
   df = pd.DataFrame(soup_data)
   df


,時間,日期,天氣狀況,溫度(℃),溫度(℉),體感溫度(℃),體感溫度(℉),降雨機率,相對溼度,蒲福風級,風速(m/s),風向,舒適度
0,00:00,09/21星期四,晴,27,81,31,88,0%,79%,65%,1,偏西風,舒適
1,03:00,09/21星期四,晴,26,79,30,86,0%,80%,65%,1,西南風,舒適
2,06:00,09/21星期四,晴,26,79,29,84,10%,76%,65%,1,偏西風,舒適
3,09:00,09/21星期四,多雲,30,86,34,93,10%,66%,65%,2,偏北風,舒適
4,12:00,09/21星期四,午後短暫雷陣雨,33,91,38,100,30%,66%,65%,1,偏北風,悶熱
5,15:00,09/21星期四,多雲,33,91,38,100,30%,72%,65%,1,偏東風,悶熱
6,18:00,09/21星期四,晴,29,84,33,91,20%,66%,65%,1,東北風,舒適
7,21:00,09/21星期四,晴,28,82,31,88,20%,75%,65%,2,偏東風,舒適
8,00:00,09/22星期五,晴,27,81,31,88,20%,77%,65%,1,偏東風,舒適
9,03:00,09/22星期五,晴,26,79,30,86,20%,82%,65%,1,東北風,舒適


In [12]:
    # 將日期拆分成日期、星期二欄
    df["星期"] = df['日期'].str.partition("星期")[1] + df['日期'].str.partition("星期")[2]
    df["日期"] = df['日期'].str.partition("星期")[0]
    df


,時間,日期,天氣狀況,溫度(℃),溫度(℉),體感溫度(℃),體感溫度(℉),降雨機率,相對溼度,蒲福風級,風速(m/s),風向,舒適度,星期
0,00:00,09/21,晴,27,81,31,88,0%,79%,65%,1,偏西風,舒適,星期四
1,03:00,09/21,晴,26,79,30,86,0%,80%,65%,1,西南風,舒適,星期四
2,06:00,09/21,晴,26,79,29,84,10%,76%,65%,1,偏西風,舒適,星期四
3,09:00,09/21,多雲,30,86,34,93,10%,66%,65%,2,偏北風,舒適,星期四
4,12:00,09/21,午後短暫雷陣雨,33,91,38,100,30%,66%,65%,1,偏北風,悶熱,星期四
5,15:00,09/21,多雲,33,91,38,100,30%,72%,65%,1,偏東風,悶熱,星期四
6,18:00,09/21,晴,29,84,33,91,20%,66%,65%,1,東北風,舒適,星期四
7,21:00,09/21,晴,28,82,31,88,20%,75%,65%,2,偏東風,舒適,星期四
8,00:00,09/22,晴,27,81,31,88,20%,77%,65%,1,偏東風,舒適,星期五
9,03:00,09/22,晴,26,79,30,86,20%,82%,65%,1,東北風,舒適,星期五


In [13]:
    # 轉為json回傳
    print(df.to_json(orient='records', force_ascii=False))

else:
    print('無此鄉鎮市區名稱！') 

SyntaxError: invalid syntax (1970001387.py, line 4)